In [ ]:
import os
import json
import h5py
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import shutil
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from datetime import datetime
from scipy import signal

## Slicing a session

In [14]:
def slice_tracking_data(src_path, dst_path, t_start, t_end):
    exp_start = np.loadtxt(os.path.join(src_path, 'positions.csv'), delimiter=',', skiprows=1)[0][0]
    
    for ds_name in ['positions', 'sounds', 'events']:
        filename = os.path.join(src_path, '%s.csv' % ds_name)
        
        with open(filename) as ff:
            header = ff.readline()
        data = np.loadtxt(filename, delimiter=',', skiprows=1)
        
        to_write = data[(data[:, 0] > exp_start + t_start) & (data[:, 0] < exp_start + t_end)]
        
        if ds_name == 'events':
            to_write[:, 4] -= to_write[0][4] - 1  # reduce trial number

            if to_write[0][-1] > 0:  # add a record for beginning of the trial
                record = np.array(to_write[0])
                record[-1] = 0
                record[0] = exp_start + t_start
                to_write = np.vstack([record, to_write])

            if to_write[-1][-1] > -1:  # add a record for end of the session
                record = np.array(to_write[-1])
                record[-1] = -1
                record[0] = exp_start + t_end
                to_write = np.vstack([to_write, record])        
        
        np.savetxt(os.path.join(dst_path, '%s.csv' % ds_name), to_write, \
                   delimiter=',', fmt='%.4f', header=header.replace('\n', ''))

In [15]:
def slice_dat_file(src_path, dst_path, duration, sampling=30000, channels=64, databyte=2):  # int16
    try:
        dat_fname = [x for x in os.listdir(src_path) if x.endswith('.dat')][0]
    except IndexError:
        print('Not .dat file found')
        return
    
    in_path  = os.path.join(src_path, dat_fname)
    out_path = os.path.join(dst_path, dat_fname)
    
    chunk_size = databyte * channels * sampling

    with open(in_path, 'rb') as in_file, open(out_path, 'wb') as out_file:
        out_file.write(in_file.read(chunk_size * duration))

In [16]:
def slice_video(src_path, dst_path, t_start, t_end):
    src_fname = os.path.join(src_path, 'video.avi')
    dst_fname = os.path.join(dst_path, 'video.avi')
    ffmpeg_extract_subclip(src_fname, t_start, t_end, targetname=dst_fname)

In [18]:
def slice_session(src_path, dst_path, t_start, t_end):
    """
    src_path       absolute path to the source session folder
    dst_path       absolute path to the destination session folder
    t_start        start time (relative to the session start)
    t_end          end time (relative to the session start)
    """
    filebase = os.path.basename(src_path)
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
    
    # first slice raw tracking data
    slice_tracking_data(src_path, dst_path, t_start, t_end)

    # second slice ephys .dat file
    slice_dat_file(src_path, dst_path, t_end - t_start)
    
    # third slice the video
    slice_video(src_path, dst_path, t_start, t_end)
    
    # last - copy .xml and .json
    for ext in ('json', 'xml'):
        src_fname = os.path.join(src_path, '%s.%s' % (filebase, ext))
        dst_fname = os.path.join(dst_path, '%s.%s' % (filebase, ext))

        shutil.copyfile(src_fname, dst_fname)